In [1]:
# word2vec和GloVe都没有对词的内部结构进行探讨
# help/helping/helped 是不是应该有所关联？

##### fastText 模型

In [1]:
# 为了使用形态信息，fastText模型提出了一种子词嵌入方法，其中子词是一个字符n-gram
# fastText可以被认为是子词级跳元模型，而非学习词级向量表示，其中每个中心词由其子词级向量之和表示。
# 旨在解决 OOV 问题 out of vocab
# 比如 n-gram n = 3 <where> = <wh , whe, her, ere , er>
# 但是这样分，词量大，语义稀疏，复杂度很高

In [2]:
# 所有提取的子词都必须是指定的长度，比如3-6，因此词表大小不能被预定义
# 出现了 字节对编码 BPE byte pair encoding的压缩算法
# 在一堆文本里找出“最常一起出现的两个东西”，把它们合成一个新东西
# 比如 playing played player 被分成 paly ing ed er
# byte pair的意思是一对相邻的最小单位

# n-gram是在用滑窗找出所有可能，BPE则是统计驱动和逐步合并
# n-gram ： 特征工程 + embedding
# BPE : 分词 + tokenization
# fasttext 和 BPE 是在 n-gram 思想下的不同处理方式

In [3]:
import collections

symbols = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
           'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
           '_', '[UNK]']

In [5]:
raw_token_freqs = {'fast_': 4, 'faster_': 3, 'tall_': 5, 'taller_': 4}
token_freqs = {}
for token, freq in raw_token_freqs.items():
    token_freqs[(' '.join(list(token)))] = raw_token_freqs[token]
token_freqs

{'f a s t _': 4, 'f a s t e r _': 3, 't a l l _': 5, 't a l l e r _': 4}

In [6]:
# 我们定义以下get_max_freq_pair函数
# 其返回词内最频繁的连续符号对，其中词来自输入词典token_freqs的键

In [7]:
def get_max_freq_pair(token_freqs):
    pairs = collections.defaultdict(int)
    for token, freq in token_freqs.items():
        symbols = token.split()
        for i in range(len(symbols) - 1):
            # “pairs”的键是两个连续符号的元组
            pairs[symbols[i], symbols[i + 1]] += freq
    return max(pairs, key=pairs.get)  # 具有最大值的“pairs”键

In [8]:
# 字节对编码将使用以下merge_symbols函数来合并最频繁的连续符号对以产生新符号。

In [9]:
def merge_symbols(max_freq_pair, token_freqs, symbols):
    symbols.append(''.join(max_freq_pair))
    new_token_freqs = dict()
    for token, freq in token_freqs.items():
        new_token = token.replace(' '.join(max_freq_pair),
                                  ''.join(max_freq_pair))
        new_token_freqs[new_token] = token_freqs[token]
    return new_token_freqs

In [10]:
num_merges = 10
for i in range(num_merges):
    max_freq_pair = get_max_freq_pair(token_freqs)
    token_freqs = merge_symbols(max_freq_pair, token_freqs, symbols)
    print(f'合并# {i+1}:',max_freq_pair)

合并# 1: ('t', 'a')
合并# 2: ('ta', 'l')
合并# 3: ('tal', 'l')
合并# 4: ('f', 'a')
合并# 5: ('fa', 's')
合并# 6: ('fas', 't')
合并# 7: ('e', 'r')
合并# 8: ('er', '_')
合并# 9: ('tall', '_')
合并# 10: ('fast', '_')


In [11]:
print(symbols)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '_', '[UNK]', 'ta', 'tal', 'tall', 'fa', 'fas', 'fast', 'er', 'er_', 'tall_', 'fast_']


In [12]:
print(list(token_freqs.keys()))

['fast_', 'fast er_', 'tall_', 'tall er_']


In [13]:
# 请注意，字节对编码的结果取决于正在使用的数据集。
# 我们还可以使用从一个数据集学习的子词来切分另一个数据集的单词。
# 作为一种贪心方法，下面的segment_BPE函数尝试将单词从输入参数symbols分成可能最长的子词。

In [14]:
def segment_BPE(tokens, symbols):
    outputs = []
    for token in tokens:
        start, end = 0, len(token)
        cur_output = []
        # 具有符号中可能最长子字的词元段
        while start < len(token) and start < end:
            if token[start: end] in symbols:
                cur_output.append(token[start: end])
                start = end
                end = len(token)
            else:
                end -= 1
        if start < len(token):
            cur_output.append('[UNK]')
        outputs.append(' '.join(cur_output))
    return outputs

In [15]:
tokens = ['tallest_', 'fatter_']
print(segment_BPE(tokens, symbols))

['tall e s t _', 'fa t t er_']
